In [18]:
from keras.models import Sequential
from keras.layers import Input,Conv2D,Lambda,LeakyReLU,Activation,Concatenate,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.applications.mobilenet import MobileNet
#import tensorflow as tf
from keras.models import Model
from tensorflow.keras import metrics
import numpy as np #矩阵库
import cv2         #图像处理库
import matplotlib #绘图库
import pandas as pd      #数据处理库
import os
import sys
import random
from datetime import datetime
from subprocess import check_output

ImageValData={}
def loadTrainData(trainDir): #读取训练图像数据及处理
    global ImageValData
    
    images=os.listdir(trainDir)
    print("验证集图片共有"+str((len(images)))+"张")
    
    for imageFileName in images:
        imageFullPath=os.path.join(trainDir,imageFileName)
        img=cv2.imread(imageFullPath)
        arr=img
        #print(arr.shape)
          #提取各颜色空间并生成LGI颜色空间
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
        Blue=arr[:,:,0]
        Green=arr[:,:,1]
        Red=arr[:,:,2]
        #print(Green.shape)
        Green=clahe.apply(Green)
        #print(Green.shape)
        Lab=cv2.cvtColor(arr,cv2.COLOR_BGR2Lab)
        Light=Lab[:,:,0]
        Light=clahe.apply(Light)
        #print(Light.shape)
        I1=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
        #print(I1.shape)
        I1=clahe.apply(I1)
        arr=cv2.merge([Green,Light,I1])
        dim0=arr.shape[0]#高度
        dim1=arr.shape[1]#宽度
        dim2=arr.shape[2]#深度
        #调整图片尺寸统一
        if (dim0<128 or dim1<128 or dim2<3):
            print("图片错误"+str(arr.shape))
        arr=cv2.resize(arr,(128,128))
        dim0=arr.shape[0]#新高度
        dim1=arr.shape[1]#新宽度
        dim2=arr.shape[2]#新深度
        #确定修改完成
        if (dim0!=128 or dim1!=128 or dim2!=3):
            print("图片错误"+str(arr.shape))
        #print(arr)
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpg','')
        ImageValData [str(imageFileName)] = np.array(arr)
    print('完成')
    return

print("读取数据"+str(datetime.now()))
sys.stdout.flush()
loadTrainData("a. Training Set")

读取数据2020-03-20 16:32:46.710765
验证集图片共有413张
完成


In [19]:
def loadValLabels():#读取标签
    raw_df=pd.read_csv('a. IDRiD_Disease Grading_Training Labels.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df
Val_label=loadValLabels()
Val_keepImages =list(ImageValData.keys())
print("完成")

共有413数据
拥有2特征
特征分为:['Image name', 'Retinopathy grade']
完成


In [20]:
val_image_arr=[]
val_data_arr=[]

for index, row in Val_label.iterrows():
    key = str(row[0])
    if key in ImageValData:
        val_image_arr.append(key)
        val_data_arr.append(np.array(ImageValData [key]))
Val_label2 = pd.DataFrame({'Image name': val_image_arr, 'data': val_data_arr})

Val_label = pd.merge(Val_label2, Val_label, left_on='Image name', right_on='Image name', how='outer')
label_header_list2 = list(Val_label.columns)

print(label_header_list2)
print('测试集有', str(len(Val_label))+"张图片")
print("完成")

['Image name', 'data', 'Retinopathy grade']
测试集有 413张图片
完成


In [21]:
valset = Val_label.reset_index(drop=True)
print(valset.head())
print("完成")

  Image name                                               data  \
0  IDRiD_001  [[[0.011764705882352941, 0.011764705882352941,...   
1  IDRiD_002  [[[0.03137254901960784, 0.0196078431372549, 0....   
2  IDRiD_003  [[[0.011764705882352941, 0.011764705882352941,...   
3  IDRiD_004  [[[0.011764705882352941, 0.011764705882352941,...   
4  IDRiD_005  [[[0.011764705882352941, 0.011764705882352941,...   

   Retinopathy grade  
0                  3  
1                  3  
2                  2  
3                  3  
4                  4  
完成


In [22]:
valX = valset['data']
valY = valset['Retinopathy grade']
name = valset['Image name']
print('训练集为', valX.shape[0])

训练集为 413


In [25]:
from keras.models import load_model
model = load_model('MobileNet.h5')
T=0
TP=0
TN=0
FN=0
FP=0
pre=np.zeros(413)
for i in range(413):
    arr = np.expand_dims(valX[i], axis=0)
    preds = model.predict(arr)
    if (preds[0,0]>preds[0,1])&(preds[0,0]>preds[0,2])&(preds[0,0]>preds[0,3])&(preds[0,0]>preds[0,4]):
        pre[i]=0
    elif (preds[0,1]>preds[0,0])&(preds[0,1]>preds[0,2])&(preds[0,1]>preds[0,3])&(preds[0,1]>preds[0,4]):
        pre[i]=1
    elif (preds[0,2]>preds[0,0])&(preds[0,2]>preds[0,1])&(preds[0,2]>preds[0,3])&(preds[0,2]>preds[0,4]):
        pre[i]=2
    elif (preds[0,3]>preds[0,0])&(preds[0,3]>preds[0,1])&(preds[0,3]>preds[0,2])&(preds[0,3]>preds[0,4]):
        pre[i]=3
    elif (preds[0,4]>preds[0,0])&(preds[0,4]>preds[0,1])&(preds[0,4]>preds[0,2])&(preds[0,4]>preds[0,3]):
        pre[i]=4


for i in range(413):
    if (pre[i]==0)&(valY[i]!=0):
        FN=FN+1
    elif (pre[i]==0)&(valY[i]==0):
        TN=TN+1
    elif (pre[i]!=0)&(valY[i]==0):
        FP=FP+1
    elif (pre[i]!=0)&(valY[i]!=0):
        TP=TP+1
        
for i in range(413):
    if pre[i]==valY[i]:
        T=T+1
        
        
print('TN:'+str(TN))
print('TP:'+str(TP))
print('FN:'+str(FN))
print('FP:'+str(FP))
P=TP+FN
print('准确率为'+str(T/413))
print('召回率为'+str(TP/P))

TN:134
TP:279
FN:0
FP:0
准确率为0.9854721549636803
召回率为1.0


In [6]:
        
for i in range(100):
    if pre[i]==valY[i]:
        T=T+1
print('TN:'+str(TN))
print('TP:'+str(TP))
print('FN:'+str(FN))
print('FP:'+str(FP))
P=TP+FN
print('准确率为'+str(T/100))
print('召回率为'+str(TP/P))

TN:45
TP:9
FN:9
FP:37
准确率为0.54
召回率为0.5
